In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
df = pd.read_csv("F:\CSV File\Thyroid_Diff.csv")

In [5]:
df.sample(6)

,Age,Gender,Smoking,Hx Smoking,Hx Radiothreapy,Thyroid Function,Physical Examination,Adenopathy,Pathology,Focality,Risk,T,N,M,Stage,Response,Recurred
8,51,F,No,No,No,Euthyroid,Single nodular goiter-right,No,Micropapillary,Uni-Focal,Low,T1a,N0,M0,I,Excellent,No
168,21,F,No,No,No,Euthyroid,Single nodular goiter-right,No,Papillary,Uni-Focal,Low,T2,N0,M0,I,Excellent,No
90,18,F,No,No,No,Euthyroid,Single nodular goiter-right,Right,Papillary,Uni-Focal,Intermediate,T1b,N1b,M0,I,Structural Incomplete,Yes
245,34,F,No,No,No,Euthyroid,Multinodular goiter,Bilateral,Papillary,Uni-Focal,Intermediate,T3a,N1b,M0,I,Indeterminate,No
312,27,M,No,No,No,Euthyroid,Multinodular goiter,Bilateral,Papillary,Multi-Focal,Intermediate,T3a,N1b,M0,I,Structural Incomplete,Yes
379,81,M,Yes,No,Yes,Euthyroid,Multinodular goiter,Extensive,Papillary,Multi-Focal,High,T4b,N1b,M1,IVB,Structural Incomplete,Yes


In [6]:
df.isnull().sum()

Age                     0
Gender                  0
Smoking                 0
Hx Smoking              0
Hx Radiothreapy         0
Thyroid Function        0
Physical Examination    0
Adenopathy              0
Pathology               0
Focality                0
Risk                    0
T                       0
N                       0
M                       0
Stage                   0
Response                0
Recurred                0
dtype: int64

In [7]:
for var in df.select_dtypes(include='object').columns:
    print(var , df[var].nunique())

Gender 2
Smoking 2
Hx Smoking 2
Hx Radiothreapy 2
Thyroid Function 5
Physical Examination 5
Adenopathy 6
Pathology 4
Focality 2
Risk 3
T 7
N 3
M 2
Stage 5
Response 4
Recurred 2


In [26]:
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, chi2

In [9]:
X = df.drop(columns=['Recurred'])
y = df['Recurred']

In [10]:
X_train , X_test, y_train , y_test = train_test_split(X,y, test_size=0.2, random_state=42)

In [15]:
trf1 = ColumnTransformer([
    ('oh', OneHotEncoder(sparse=False, handle_unknown='ignore', drop='first'), [1, 2,3, 4,5,6,7,8,9,10,11,12,13,14,15])
], remainder='passthrough')
trf2 = ColumnTransformer([
    ('Scaling', MinMaxScaler(), slice(0,41))
])

In [16]:
trf3 = SelectKBest(score_func=chi2, k = 10)

In [17]:
trf4 = DecisionTreeClassifier()

In [18]:
pipe = Pipeline([
    ('trf1', trf1),
    ('trf2', trf2),
    ('trf3', trf3),
    ('trf4', trf4)
])

In [19]:
le  = LabelEncoder()
le.fit(y_train)

LabelEncoder()

In [21]:
le  = LabelEncoder()
le.fit(y_train)

LabelEncoder()

In [22]:
pipe.fit(X_train, y_train)

D:\AnaConda\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('oh',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore',
                                                                sparse=False),
                                                  [1, 2, 3, 4, 5, 6, 7, 8, 9,
                                                   10, 11, 12, 13, 14, 15])])),
                ('trf2',
                 ColumnTransformer(transformers=[('Scaling', MinMaxScaler(),
                                                  slice(0, 41, None))])),
                ('trf3',
                 SelectKBest(score_func=<function chi2 at 0x0000024F294D4A40>)),
                ('trf4', DecisionTreeClassifier())])

In [23]:
y_pred  = pipe.predict(X_test)
accuracy_score(y_test, y_pred)

0.948051948051948

In [25]:
cross_val_score(pipe, X_train , y_train, cv = 5 , scoring = 'accuracy').mean()

D:\AnaConda\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
D:\AnaConda\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
D:\AnaConda\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [6] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
D:\AnaConda\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
D:\AnaCo

0.9345848757271285

In [34]:
params = {
    'trf4__max_depth': [1,2,3,4,5,None]
}
pipe.named_steps

{'trf1': ColumnTransformer(remainder='passthrough',
                   transformers=[('oh',
                                  OneHotEncoder(drop='first',
                                                handle_unknown='ignore',
                                                sparse=False),
                                  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14,
                                   15])]),
 'trf2': ColumnTransformer(transformers=[('Scaling', MinMaxScaler(),
                                  slice(0, 41, None))]),
 'trf3': SelectKBest(score_func=<function chi2 at 0x0000024F294D4A40>),
 'trf4': DecisionTreeClassifier()}

In [35]:
gird = GridSearchCV(pipe, params, cv = 5 , scoring = 'accuracy')
gird.fit(X_train , y_train)

D:\AnaConda\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
D:\AnaConda\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
D:\AnaConda\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [6] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
D:\AnaConda\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
D:\AnaCo

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('trf1',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('oh',
                                                                         OneHotEncoder(drop='first',
                                                                                       handle_unknown='ignore',
                                                                                       sparse=False),
                                                                         [1, 2,
                                                                          3, 4,
                                                                          5, 6,
                                                                          7, 8,
                                                                          9, 10,
                                                                          11,
                                                                          12,
                                                                          13,
                                                                          14,
                                                                          15])])),
                                       ('trf2',
                                        ColumnTransformer(transformers=[('Scaling',
                                                                         MinMaxScaler(),
                                                                         slice(0, 41, None))])),
                                       ('trf3',
                                        SelectKBest(score_func=<function chi2 at 0x0000024F294D4A40>)),
                                       ('trf4', DecisionTreeClassifier())]),
             param_grid={'trf4__max_depth': [1, 2, 3, 4, 5, None]},
             scoring='accuracy')

In [36]:
gird.best_score_

0.9411951348492862

In [37]:
gird.best_params_

{'trf4__max_depth': 1}